# ***GİRİŞ*** 

*Bu proje, X platformundan (Twitter) "Suriye" kelimesi içeren tweet’lerden yararlanarak, son zamanlarda gündemdeki olaylara insanların tepkilerini analiz etmeyi amaçlamaktadır. Proje, Suriye ile ilgili yapılan paylaşımların duygu analizini yaparak, toplumun bu konuda genel olarak olumlu ya da olumsuz bir bakış açısına sahip olup olmadığını ortaya koymayı hedeflemektedir. Veri bilimi perspektifinden ele alınan bu analiz, tamamen teknik bir çalışmadır ve herhangi bir siyasi tartışma amacı gütmemektedir.*


In [1]:
#Ana kütüphaneler
import numpy as np
import pandas as pd
import os
import re

#Veri görselleştirme kütüphanleri
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#Veri işlemek için kütüphaneler
import nltk
from nltk.corpus import stopwords

#Sentiment skorları için kütüphaneler
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

/Users/ykoc7/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


# ***Veri Yükleme ve İlk İnceleme***


In [2]:
#Dosyayı dataframe'e aktarıyoruz
df = pd.read_csv("tweets.csv")
df.head(3)

,Tweets
0,Suriye’de Esad’ın gidişini kutluyorlar ama yer...
1,Muhaliflerin Suriye’de 9 günlük ilerleyişi.
2,"""Yemin olsun sabahleyin ansızın baskın yapanla..."


# ***Verinin Temizlenmesi***


In [3]:
#Türkçe stopwordsler
nltk.download('stopwords')
stopwords_set = set(stopwords.words('turkish'))
#Emojileri temizlemek için bir fonksiyon
def remove_emojis(text):
    return re.sub(r"[\U00010000-\U0010FFFF]", "", text)

# Tweet temizleme fonksiyonu
def clean_tweet(tweet):
    if not tweet: return ""
    tweet = re.sub(r'rt @[A-Za-z0-9_]+|@[A-Za-z0-9_]+|#[A-Za-z0-9_]+|http\S+|[()!?|:,.;-_\""]', " ", tweet.lower())
    tweet = re.sub(r'\[.*?\]', ' ', remove_emojis(tweet))
    return " ".join([word for word in tweet.split() if word not in stopwords_set and len(word) > 1])

#Temizleme işlemi
df['Clean_Tweets'] = df['Tweets'].apply(clean_tweet).dropna()
df.head(3)


[nltk_data] Downloading package stopwords to /Users/ykoc7/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Tweets,Clean_Tweets
0,Suriye’de Esad’ın gidişini kutluyorlar ama yer...,suriye’de esad’ın gidişini kutluyorlar yerine ...
1,Muhaliflerin Suriye’de 9 günlük ilerleyişi.,muhaliflerin suriye’de günlük ilerleyişi
2,"""Yemin olsun sabahleyin ansızın baskın yapanla...",yemin olsun sabahleyin ansızın baskın yapanlar...


# ***Lemmatization İşlemi***


In [4]:
#Lemmatizer
%pip install stanza
import stanza

Note: you may need to restart the kernel to use updated packages.


: 

In [ ]:
#NOT: Bu kısım biraz uzun sürebilir mümkünse GPU kullanmanızı öneririm
#Türkçe modeli indiriyoruz
stanza.download('tr')

#Stanza Türkçe NLP pipeline'ı
nlp = stanza.Pipeline('tr', processors='tokenize,pos,lemma')

def stanza_lemmatizer_to_list(df_column):
    lemmatized_words = []
    exclude_suffixes = ["ki", "i", "li"]  #Stanzanın sorun yaşadağı ekleri burada belirtiyoruz
    for text in df_column:
        doc = nlp(text)
        for sentence in doc.sentences:
            for word in sentence.words:
                lemma = word.lemma if word.lemma is not None else word.text
                if any(lemma.endswith(suffix) for suffix in exclude_suffixes):
                    continue
                lemmatized_words.append(lemma)
    return lemmatized_words

#Lemmatize fonksiyonunu uygulama
words = stanza_lemmatizer_to_list(df['Clean_Tweets'])

2025-01-06 23:17:39 INFO: Downloaded file to /Users/ykoc7/stanza_resources/resources.json
2025-01-06 23:17:39 INFO: Downloading default packages for language: tr (Turkish) ...
2025-01-06 23:17:39 INFO: File exists: /Users/ykoc7/stanza_resources/tr/default.zip
2025-01-06 23:17:41 INFO: Finished downloading models and saved to /Users/ykoc7/stanza_resources
2025-01-06 23:17:41 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-01-06 23:17:41 INFO: Downloaded file to /Users/ykoc7/stanza_resources/resources.json
2025-01-06 23:17:41 WARNING: Language tr package default expects mwt, which has been added
2025-01-06 23:17:42 INFO: Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| mwt       | imst          |
| pos       | imst_charlm   |
| lemma     | imst_nocharlm |

2025-01-06 23:17:42 INFO: Using device: cpu
2025-01-06 23:17:42 INFO: Loading: tokenize
2025-01-06 23:17:42 INFO: Loading: mwt


# ***Bigram ve Trigram Analizi***


In [ ]:
#Bigrams (iki kelimeden oluşan gruplar) serisini oluştur ve en sık geçen 20 bigramı seç
bigrams_series = pd.Series(nltk.ngrams(words, 2)).value_counts()[:20]

#Bigram'ları görselleştirme
plt.figure(figsize=(12, 8))
bigrams_series.sort_values().plot.barh(color='#a557ff', width=0.85, edgecolor='black')
plt.title('En Sık Geçen 20 Bigram', fontsize=16, fontweight='bold')
plt.ylabel('Bigram', fontsize=12)
plt.xlabel('Görülme Sayısı', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

#Trigrams (üç kelimeden oluşan gruplar) serisini oluştur ve en sık geçen 20 trigramı seç
trigrams_series = pd.Series(nltk.ngrams(words, 3)).value_counts()[:20]

#Trigram'ları görselleştirme
plt.figure(figsize=(12, 8))
trigrams_series.sort_values().plot.barh(color='#f5a22f', width=0.85, edgecolor='black')
plt.title('En Sık Geçen 20 Trigram', fontsize=16, fontweight='bold')
plt.ylabel('Trigram', fontsize=12)
plt.xlabel('Görülme Sayısı', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

# ***WordCloud Oluşturma***


In [ ]:
#Wordcloud oluşturma fonksiyonu
def generate_word_cloud(text):
    wordcloud = WordCloud(
        width = 3000,
        height = 2000,
        background_color = 'black',
        colormap = 'plasma').generate(str(text))
    fig = plt.figure(
        figsize = (40, 30),
        facecolor = 'k',
        edgecolor = 'k')
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.show()
    
#Wordcloud
generate_word_cloud(words)

# ***Sentiment Analizi ve Görselleştirme***


In [ ]:
#Model ve tokenizer yükleme
model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")

#Pipeline oluşturma
sentiment_analysis = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

#Sentiment analizi fonksiyonu
def get_sentiment_analysis(tweet):
    """Cümle pozitifse True, değilse False."""
    p = sentiment_analysis(tweet)
    return pd.Series([p[0]["label"], round(p[0]["score"], 4)])

#Fonksiyonu uygulama
df["sentiment_label"] = "-"
df["sentiment_score"] = -1
df[["sentiment_label", "sentiment_score"]] = df.apply(lambda row: get_sentiment_analysis(row['Clean_Tweets']), axis=1)

#Görsel boyutu
plt.rcParams['figure.figsize'] = (8, 5)
fig, ax = plt.subplots()
plt.rcdefaults()

#Her bir sentiment için barları çizme
bars = plt.bar(df["sentiment_label"].unique(), df["sentiment_label"].value_counts(), color=['#389900', '#002199'])

#Grafik düzenlemeleri
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')

ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)

#Barların üst kısmına metin ek açıklamaları
for bar in bars:
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.9,
        round(bar.get_height(), 1),
        horizontalalignment='center',
        color='black',
        weight='bold'
    )

#Grafik şeklini iyileştirme
fig.tight_layout()
plt.xlabel('Sentiment', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Sentiment Distribution', fontsize=14)

#Grafik gösterimi
plt.show()

In [ ]:
# Sentiment label'ların frekanslarını al
sentiment_counts = df["sentiment_label"].value_counts()
#Pie chart için verileri hazırlama
labels = sentiment_counts.index
sizes = sentiment_counts.values

#Pie chart oluşturma
fig, ax = plt.subplots(figsize=(8, 8))
ax.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=140, colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
ax.axis('equal')  # Daire şeklinde yapmak için

#Grafik başlığı
plt.title("Sentiment Dağılımı", fontsize=14, pad=20)

#Grafik gösterimi
plt.tight_layout()
plt.show()